# Needed imports
Importing all the packages we need for the data we'll be using

In [16]:
# web scraping
from bs4 import BeautifulSoup
import requests

# doing certain tasks in parallel
from concurrent.futures import ThreadPoolExecutor

# for general os operations
import os

# for unzipping zip files
import zipfile

# for data manipulation
import pandas as pd
import numpy as np
from datetime import datetime

# for database storage
import sqlite3

# for json work
import json

In [17]:
# below is the base url where all the data lives
url = 'https://s3.amazonaws.com/tripdata/'

response = requests.get(url)

soup = BeautifulSoup(response.text, 'xml')

all_data_files = soup.find_all('Key')

print(f'There are {len(all_data_files)} total files.')

There are 227 total files.


In [18]:
# Set the path to save csv files
csv_save_path = 'data/raw/'

In [19]:
# Let's check to see if the database exists. If so, let's find the latest bit of data and ensure that we're not redownloading processed data
database_path = 'data/database/citibike_data.db'

database_exists = False

if os.path.exists(database_path):
    database_exists = True
    
    with sqlite3.connect(database_path) as connection:
        
        # Create a cursor object
        cursor = connection.cursor()
        
        # Execute the SQL query to get the maximum date table
        max_year_query = """
        SELECT 
            max(cast(yr as INTEGER)) as max_yr
        FROM (
            SELECT SUBSTR(name, -4) as yr 
            FROM sqlite_master 
            WHERE 
                type='table'            
        )
        ;
        """
        cursor.execute(max_year_query)

        # Fetch the result
        max_year_table = cursor.fetchone()[0]

        # Execute the query to get the max date
        max_date_query = f"""
        select max(started_at) as max_started_at
        from citibike_rides_{max_year_table}
        """
        cursor.execute(max_date_query)

        # Fetch the result
        max_database_date = datetime.strptime(cursor.fetchone()[0], "%Y-%m-%d %H:%M:%S")
        
        
    print(f"The SQLite database at '{database_path}' exists and the max start_at is {max_database_date}")

The SQLite database at 'data/database/citibike_data.db' exists and the max start_at is 2023-11-30 23:59:57


In [20]:
# Create a function to download files
def download_and_unzip_file(url, save_path):
    
    # Download the file
    r = requests.get(url)
    with open(save_path, 'wb') as file:
        file.write(r.content)

    # Extract the CSV from the downloaded zip file
    with zipfile.ZipFile(save_path, 'r') as zipped_file:
        csv_filename = [name for name in zipped_file.namelist()]
        zipped_file.extract(csv_filename[0], os.path.dirname(save_path))
    
    # Delete the zip file
    os.remove(save_path)

    print(f'\tCompleted {csv_filename[0]}')
    

# Create a function that'll download each file in parallel.
#   This takes a list of file URLs and a common save directory.
def download_files_in_parallel(file_urls, save_dir):
    total_system_workers = os.cpu_count() or 1
    usable_workers = max(1, total_system_workers)
    with ThreadPoolExecutor(max_workers=usable_workers) as executor:
        # Use executor.map to apply the download_file function to each URL
        # Use save_dir as the common save directory for all files
        executor.map(lambda url: download_and_unzip_file(url, os.path.join(save_dir, os.path.basename(url))), file_urls)

# Initialize a list of the files we'll want to download.
files_to_download = []

# Here we'll only grab NYC rides. We'll do this by first checking that the file is a .zip file and then ensuring that it doesn't contain 'JC' in the title.
for each_file in all_data_files:
    temp_file_name  = each_file.get_text()
    temp_filetype   = temp_file_name[-4:]
    
    if temp_filetype == '.zip' and not('JC' in temp_file_name):
        
        # We'll also ensure that the file contains new data by comparing the months only if the database exists.
        if database_exists:
            if datetime.strptime(temp_file_name[:6], '%Y%m') >= max_database_date:
                files_to_download.append(url + temp_file_name)
        else:
            files_to_download.append(url + temp_file_name)

print(f'{len(files_to_download)} file(s) to download.')

download_files_in_parallel(
    files_to_download,
    csv_save_path
)


0 file(s) to download.


In [21]:
# With this data, we need to make sure that column names align. 
# We're going to have everything align to the most recent data file (202311-citibike-tripdata.csv)
if not database_exists:
    # This funciton will check if column names align with the expected list
    def check_column_alignment(test_df, expected_columns):
        actual_columns = test_df.columns.tolist()
        return set(actual_columns) == set(expected_columns)

    # Read in the ideal data archetecture
    ideal_df = pd.read_csv(
        csv_save_path + '/202311-citibike-tripdata.csv',
        nrows=100
    )

    # create a list of ideal columns
    ideal_columns = ideal_df.columns.tolist()

    ideal_columns

In [22]:
if not database_exists:
    # Initialize a list of non-ideal columns
    non_ideal_columnsets = []

    # Iterate through each file and document those that aren't ideal
    for file_name in os.listdir(csv_save_path):
        temp_df = pd.read_csv(
            os.path.join(csv_save_path,file_name),
            nrows=100
        )
        if not(check_column_alignment(temp_df, ideal_columns)):
            non_ideal_columnsets.append(set(temp_df.columns.tolist()))

    # Get the unique set of non_ideal_columnssets
    non_ideal_unique = list(map(set,set(map(frozenset, non_ideal_columnsets))))

    # show the unique datasets that need to be mapped
    non_ideal_unique

In [23]:
# Remind me what the ideal columns are:
if not database_exists:
    ideal_columns

In [24]:
if database_exists:
    with open('config/column_map.json', 'r') as json_file:
        column_mapping = json.load(json_file)
else:
    # Initialize an empty dictionary
    column_mapping = {}

    # Ask the user how each item should be mapped
    for each_set in non_ideal_unique:
        for each_column in each_set:
            column_mapping[each_column] = input(f'What goes in for "{each_column}"?')

In [25]:
# Save column mapping to a json in the config file
with open('config/column_map.json', 'w') as json_file:
    json.dump(column_mapping, json_file, indent=2)

column_mapping

{'end station longitude': 'end_lng',
 'stoptime': 'ended_at',
 'start station id': 'start_station_id',
 'end station latitude': 'end_lat',
 'start station name': 'start_station_name',
 'tripduration': 'trip_duration',
 'gender': 'gender',
 'start station longitude': 'start_lng',
 'starttime': 'started_at',
 'end station id': 'end_station_id',
 'end station name': 'end_station_name',
 'bikeid': 'bike_id',
 'start station latitude': 'start_lat',
 'usertype': 'member_casual',
 'birth year': 'birth_year',
 'Start Station Name': 'start_station_name',
 'End Station Latitude': 'end_lat',
 'Start Station Latitude': 'start_lat',
 'Bike ID': 'bike_id',
 'Birth Year': 'birth_year',
 'End Station Name': 'end_station_name',
 'End Station ID': 'end_station_id',
 'Start Time': 'started_at',
 'User Type': 'member_casual',
 'Gender': 'gender',
 'Start Station Longitude': 'start_lng',
 'End Station Longitude': 'end_lng',
 'Start Station ID': 'start_station_id',
 'Trip Duration': 'trip_duration',
 'Stop 

In [29]:
# Make sure that every column mapped properly

if len(os.listdir(csv_save_path)) > 0:
    sample_data = []

    for file_name in os.listdir(csv_save_path):
        temp_df = pd.read_csv(
            os.path.join(csv_save_path,file_name),
            nrows=100,
            parse_dates=True
        )

        temp_df.rename(
            columns = column_mapping,
            inplace=True
        )

        temp_df['started_at'] = pd.to_datetime(temp_df['started_at'])
        temp_df['ended_at'] = pd.to_datetime(temp_df['ended_at'])

        sample_data.append(
            temp_df
        )

    sample_df = pd.concat(sample_data)
    sample_df.info()

In [30]:
# Connect to the SQLite database (creates a new database if it doesn't exist)
if len(os.listdir(csv_save_path)) > 0:
    with sqlite3.connect(database_path) as connection:
        
        for file_name in os.listdir(csv_save_path):
            temp_save_table = f'citibike_rides_{file_name[:4]}'
            print(f'Writing "{file_name}" to "{temp_save_table}"...')
            temp_df = pd.read_csv(
                os.path.join(csv_save_path,file_name),
                header=0,
                low_memory=False
            )

            temp_df.rename(
                columns = column_mapping,
                inplace=True
            )

            t_col_list = list(temp_df.columns)

            # Set datatypes.

            temp_df['started_at'] = pd.to_datetime(temp_df['started_at'])
            temp_df['ended_at'] = pd.to_datetime(temp_df['ended_at'])

            # If a column doesn't exist, then create it with null data and the datatype from sample_df
            for each_column in list(sample_df.columns):
                if each_column not in list(temp_df.columns):
                    temp_df[each_column] = np.nan
                    temp_df[each_column] = temp_df[each_column].astype(sample_df.dtypes[each_column])

            temp_df.to_sql(
                temp_save_table,
                connection,
                index=False,
                if_exists='append'
            )

            
            connection.commit()

            

In [31]:
if len(os.listdir(csv_save_path)) > 0:
    for each_file in os.listdir(csv_save_path):
        file_path = os.path.join(csv_save_path, each_file)
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Deleted: {file_path}")

In [32]:
# lets look at the 2013 data
with sqlite3.connect(database_path) as connection:
    df = pd.read_sql_query(
        'select * from citibike_rides_2013 limit 100',
        connection
    )
df.head()

,trip_duration,started_at,ended_at,start_station_id,start_station_name,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,bike_id,member_casual,birth_year,gender,ride_id,rideable_type
0,634,2013-07-01 00:00:00,2013-07-01 00:10:34,164,E 47 St & 2 Ave,40.753231,-73.970325,504,1 Ave & E 15 St,40.732219,-73.981656,16950,Customer,\N,0,None,None
1,1547,2013-07-01 00:00:02,2013-07-01 00:25:49,388,W 26 St & 10 Ave,40.749718,-74.002950,459,W 20 St & 11 Ave,40.746745,-74.007756,19816,Customer,\N,0,None,None
2,178,2013-07-01 00:01:04,2013-07-01 00:04:02,293,Lafayette St & E 8 St,40.730287,-73.990765,237,E 11 St & 2 Ave,40.730473,-73.986724,14548,Subscriber,1980,2,None,None
3,1580,2013-07-01 00:01:06,2013-07-01 00:27:26,531,Forsyth St & Broome St,40.718939,-73.992663,499,Broadway & W 60 St,40.769155,-73.981918,16063,Customer,\N,0,None,None
4,757,2013-07-01 00:01:10,2013-07-01 00:13:47,382,University Pl & E 14 St,40.734927,-73.992005,410,Suffolk St & Stanton St,40.720664,-73.985180,19213,Subscriber,1986,1,None,None


In [33]:
# lets look at the 2022 data
with sqlite3.connect(database_path) as connection:
    df = pd.read_sql_query(
        'select * from citibike_rides_2022 limit 100',
        connection
    )
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_duration,bike_id,birth_year,gender
0,55262E4365A955A2,classic_bike,2022-01-18 08:23:52,2022-01-18 08:28:18,Boerum Pl\t& Pacific St,4488.09,Clinton St & Joralemon St,4605.04,40.688489,-73.991160,40.692395,-73.993379,member,None,None,None,None
1,D272F1B15D841EC0,classic_bike,2022-01-21 09:03:22,2022-01-21 09:05:44,E 12 St & Ave C,5616.08,E 10 St & Avenue A,5659.05,40.727243,-73.976831,40.727408,-73.981420,member,None,None,None,None
2,D1FCEF55EB4A807F,classic_bike,2022-01-22 14:28:32,2022-01-22 14:53:18,W 21 St & 6 Ave,6140.05,W 44 St & 11 Ave,6756.05,40.741740,-73.994156,40.762009,-73.996975,member,None,None,None,None
3,E9CBDC6A0162C068,electric_bike,2022-01-19 14:49:47,2022-01-19 14:54:02,38 St & 30 Ave,6850.01,Crescent St & 30 Ave,6958.06,40.764175,-73.915840,40.768692,-73.924957,member,None,None,None,None
4,2177A5B57326CE9B,electric_bike,2022-01-16 14:36:06,2022-01-16 14:44:06,Pacific St & Nevins St,4362.04,Clinton St & Tillary St,4748.07,40.685376,-73.983021,40.696233,-73.991421,member,None,None,None,None
